In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
from itertools import combinations,permutations
from sklearn.preprocessing import StandardScaler
from time import time
from thundersvm import SVC

%matplotlib inline

In [5]:
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
feature_name = ["binary","NCPA","emb","PSNP","ENAC","PseDNC","EIIP","PCP","DBPF"]

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  #选择设备
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Auc = np.zeros(len(type_name)*len(cell_name)*5*1023).reshape(1023,-1)
Acc = np.zeros(len(type_name)*len(cell_name)*5*1023).reshape(1023,-1)
Mcc = np.zeros(len(type_name)*len(cell_name)*5*1023).reshape(1023,-1)

col_name = []
row_name = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        for i in range(5):
            col_name.append(type_name[typei]+"_"+cell_name[celli]+"_"+str(i))
            train_data = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,1:]
            test_data = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,1:]
            
            train_data = np.array(train_data)
            test_data = np.array(test_data)
            
            x = list(range(10))
            fea_list = ['binary','NCPA','emb','PSNP','ENAC','PseDNC','EIIP','PCP','DBPF','Blastn']
            for feai in range(len(fea_list)):
                #读th文件，再找对应type跟cell和KFoldi的行，找到最后一个值，然后现在的值减去这个最后一个值
                if fea_list[feai] == 'Blastn':
                    Blastn_data = pd.read_csv("Blastn/"+type_name[typei]+"_"+cell_name[celli]+"_result_train.csv",header=None)
                    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics(np.array(Blastn_data.iloc[:,1]),np.array(Blastn_data.iloc[:,2]))
                    train_data[:,feai] = train_data[:,feai]-th
                    test_data[:,feai] = test_data[:,feai]-th
                else:
                    th_list = pd.read_csv("out/th_"+fea_list[feai]+".csv")
                    th = np.float(th_list[th_list['0'] == type_name[typei]+"_"+cell_name[celli]+"_"+fea_list[feai]+"_BILSTM+Self-Attention_KFold_"+str(i)].iloc[:,-1])
                    train_data[:,feai] = train_data[:,feai]-th
                    test_data[:,feai] = test_data[:,feai]-th
                    
            row = -1
            for j in range(10):
                for com_one in list(combinations(x, j+1)):
                    x_train = []
                    x_test = []
                    for m in com_one:
                        row_name += fea_list[m]+"_"
                        x_train.append(train_data[:,m:m+1])
                        x_test.append(test_data[:,m:m+1])
                    row+=1
                    model = SVC(kernel = "polynomial",gamma="auto", degree = 1, cache_size=7000)
                    clf = model.fit(np.array(x_train).reshape(len(com_one),-1).transpose(), train_data[:,-1])
                    th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(np.array(x_train).reshape(len(com_one),-1).transpose()).reshape(-1),train_data[:,-1].reshape(-1))                
                    _,_,_,_,_,_,acc,mcc,auc = metricsCal.get_test_metrics(clf.decision_function(np.array(x_test).reshape(len(com_one),-1).transpose()).reshape(-1),test_data[:,-1].reshape(-1),th)
                    Auc[row][typei*30+celli*5+i]=auc
                    Acc[row][typei*30+celli*5+i]=acc
                    Mcc[row][typei*30+celli*5+i]=mcc

In [8]:
#获取行名
x = list(range(10))
row_name = []
fea_list = ['binary','NCPA','emb','PSNP','ENAC','PseDNC','EIIP','PCP','DBPF','Blastn']
for j in range(10):
    for com_one in list(combinations(x, j+1)):
        row_one = ""
        for m in com_one:
            row_one += fea_list[m]+"_"
        row_one = row_one[0:-1]
        row_name.append(row_one)

In [9]:
len(row_name)

1023

In [21]:
#添加第一列
Auc_col = np.append(np.array(row_name).reshape(-1,1),Auc,axis=1)
#补全header
col_kk = col_name
col_kk.insert(0,"row_name")
#保存文件
pd.DataFrame(Auc_col).to_csv("Stacking_result/Poly_10_Standard.csv",header = col_kk,index=0)
#读取文件
data = pd.read_csv("Stacking_result/Poly_10_Standard.csv",index_col = "row_name")
for i in range(len(np.array(data.index))):
    if np.array(data.index)[i]=="NCPA_emb_PSNP_PCP_Blastn":
        print(i)

524


In [15]:
np.mean(np.array(data)[524])

0.7877487439576926

In [22]:
np.mean(np.array(data)[524])

0.7877486774758737